<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Oliver", "transactions": [{"transaction-id": 559, "amount": 5943}, {"transaction-id": 5253, "amount": 5080}]}\n',)

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Oliver", "transactions": [{"transaction-id": 559, "amount": 5943}, {"transaction-id": 5253, "amount": 5080}]}\n',
 '{"id": 1, "name": "Wendy", "transactions": [{"transaction-id": 1317, "amount": 254}, {"transaction-id": 1743, "amount": 263}, {"transaction-id": 1805, "amount": 252}, {"transaction-id": 2491, "amount": 257}, {"transaction-id": 2514, "amount": 264}, {"transaction-id": 2841, "amount": 239}, {"transaction-id": 4155, "amount": 258}, {"transaction-id": 4447, "amount": 251}, {"transaction-id": 4555, "amount": 261}, {"transaction-id": 5717, "amount": 253}, {"transaction-id": 6254, "amount": 256}, {"transaction-id": 6282, "amount": 256}, {"transaction-id": 6459, "amount": 265}, {"transaction-id": 6602, "amount": 250}, {"transaction-id": 6657, "amount": 257}, {"transaction-id": 7305, "amount": 261}, {"transaction-id": 7941, "amount": 281}, {"transaction-id": 8274, "amount": 263}, {"transaction-id": 8429, "amount": 264}, {"transaction-id": 9005, "amount": 267},

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Oliver',
  'transactions': [{'transaction-id': 559, 'amount': 5943},
   {'transaction-id': 5253, 'amount': 5080}]},
 {'id': 1,
  'name': 'Wendy',
  'transactions': [{'transaction-id': 1317, 'amount': 254},
   {'transaction-id': 1743, 'amount': 263},
   {'transaction-id': 1805, 'amount': 252},
   {'transaction-id': 2491, 'amount': 257},
   {'transaction-id': 2514, 'amount': 264},
   {'transaction-id': 2841, 'amount': 239},
   {'transaction-id': 4155, 'amount': 258},
   {'transaction-id': 4447, 'amount': 251},
   {'transaction-id': 4555, 'amount': 261},
   {'transaction-id': 5717, 'amount': 253},
   {'transaction-id': 6254, 'amount': 256},
   {'transaction-id': 6282, 'amount': 256},
   {'transaction-id': 6459, 'amount': 265},
   {'transaction-id': 6602, 'amount': 250},
   {'transaction-id': 6657, 'amount': 257},
   {'transaction-id': 7305, 'amount': 261},
   {'transaction-id': 7941, 'amount': 281},
   {'transaction-id': 8274, 'amount': 263},
   {'transaction-id': 84

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 2,
  'name': 'Alice',
  'transactions': [{'transaction-id': 25, 'amount': 1697},
   {'transaction-id': 214, 'amount': 1661},
   {'transaction-id': 289, 'amount': 1624},
   {'transaction-id': 882, 'amount': 1649},
   {'transaction-id': 1213, 'amount': 1413},
   {'transaction-id': 1242, 'amount': 1679},
   {'transaction-id': 1262, 'amount': 1581},
   {'transaction-id': 1414, 'amount': 1506},
   {'transaction-id': 1499, 'amount': 1721},
   {'transaction-id': 1785, 'amount': 1594},
   {'transaction-id': 1899, 'amount': 1552},
   {'transaction-id': 2117, 'amount': 1753},
   {'transaction-id': 2141, 'amount': 1609},
   {'transaction-id': 2234, 'amount': 1560},
   {'transaction-id': 2310, 'amount': 1454},
   {'transaction-id': 2601, 'amount': 1596},
   {'transaction-id': 2697, 'amount': 1573},
   {'transaction-id': 2768, 'amount': 1756},
   {'transaction-id': 2784, 'amount': 1642},
   {'transaction-id': 2906, 'amount': 1536},
   {'transaction-id': 2945, 'amount': 1532},
   {'transacti

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 58},
 {'name': 'Alice', 'count': 126},
 {'name': 'Alice', 'count': 17},
 {'name': 'Alice', 'count': 58},
 {'name': 'Alice', 'count': 38})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(58, 126, 17, 58, 38)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

47.88623853211009

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 25, 'amount': 1697},
  {'transaction-id': 214, 'amount': 1661},
  {'transaction-id': 289, 'amount': 1624},
  {'transaction-id': 882, 'amount': 1649},
  {'transaction-id': 1213, 'amount': 1413},
  {'transaction-id': 1242, 'amount': 1679},
  {'transaction-id': 1262, 'amount': 1581},
  {'transaction-id': 1414, 'amount': 1506},
  {'transaction-id': 1499, 'amount': 1721},
  {'transaction-id': 1785, 'amount': 1594},
  {'transaction-id': 1899, 'amount': 1552},
  {'transaction-id': 2117, 'amount': 1753},
  {'transaction-id': 2141, 'amount': 1609},
  {'transaction-id': 2234, 'amount': 1560},
  {'transaction-id': 2310, 'amount': 1454},
  {'transaction-id': 2601, 'amount': 1596},
  {'transaction-id': 2697, 'amount': 1573},
  {'transaction-id': 2768, 'amount': 1756},
  {'transaction-id': 2784, 'amount': 1642},
  {'transaction-id': 2906, 'amount': 1536},
  {'transaction-id': 2945, 'amount': 1532},
  {'transaction-id': 2970, 'amount': 1477},
  {'transaction-id': 3021, 'amount': 

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 25, 'amount': 1697},
 {'transaction-id': 214, 'amount': 1661},
 {'transaction-id': 289, 'amount': 1624})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(1697, 1661, 1624)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

789.3863514445551

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 129), ('Alice', 130), ('Alice', 132), ('Alice', 154), ('Bob', 135), ('Bob', 135), ('Bob', 137), ('Bob', 158), ('Charlie', 96), ('Charlie', 96), ('Charlie', 96), ('Charlie', 112), ('Dan', 108), ('Dan', 108), ('Dan', 108), ('Dan', 126), ('Edith', 93), ('Edith', 97), ('Edith', 98), ('Edith', 111), ('Frank', 119), ('Frank', 120), ('Frank', 120), ('Frank', 139), ('George', 43), ('George', 45), ('George', 45), ('George', 53), ('Hannah', 168), ('Hannah', 168), ('Hannah', 168), ('Hannah', 196), ('Ingrid', 180), ('Ingrid', 209), ('Ingrid', 359), ('Jerry', 71), ('Jerry', 71), ('Jerry', 72), ('Jerry', 84), ('Kevin', 119), ('Kevin', 119), ('Kevin', 120), ('Kevin', 140), ('Laura', 84), ('Laura', 84), ('Laura', 84), ('Laura', 98), ('Michael', 117), ('Michael', 119), ('Michael', 120), ('Michael', 140), ('Norbert', 155), ('Norbert', 155), ('Norbert', 156), ('Norbert', 181), ('Oliver', 96), ('Oliver', 96), ('Oliver', 96), ('Oliver', 112), ('Patricia', 48), ('Patricia', 48), ('Patricia', 104)

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 545), ('Bob', 565), ('Charlie', 400), ('Dan', 450), ('Edith', 399), ('Frank', 498), ('George', 186), ('Hannah', 700), ('Ingrid', 748), ('Jerry', 298), ('Kevin', 498), ('Laura', 350), ('Michael', 496), ('Norbert', 647), ('Oliver', 400), ('Patricia', 200), ('Quinn', 363), ('Ray', 600), ('Sarah', 478), ('Tim', 386), ('Ursula', 384), ('Victor', 481), ('Wendy', 716), ('Xavier', 495), ('Yvonne', 750), ('Zelda', 150)]
CPU times: user 213 ms, sys: 16.9 ms, total: 230 ms
Wall time: 581 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


,id,name,transactions
0,0,Oliver,"[{'transaction-id': 559, 'amount': 5943}, {'tr..."
1,1,Wendy,"[{'transaction-id': 1317, 'amount': 254}, {'tr..."
2,2,Alice,"[{'transaction-id': 25, 'amount': 1697}, {'tra..."
3,3,Alice,"[{'transaction-id': 493, 'amount': 2095}, {'tr..."
4,4,Hannah,"[{'transaction-id': 613, 'amount': 7113}, {'tr..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 218 ms, sys: 21.5 ms, total: 239 ms
Wall time: 1.49 s


name
Alice      545
Bob        565
Charlie    400
Dan        450
Edith      399
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Oliver', 'amount': 5943, 'transaction-id': 559},
 {'id': 0, 'name': 'Oliver', 'amount': 5080, 'transaction-id': 5253},
 {'id': 1, 'name': 'Wendy', 'amount': 254, 'transaction-id': 1317})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Oliver,5943,559
1,0,Oliver,5080,5253
2,1,Wendy,254,1317
3,1,Wendy,263,1743
4,1,Wendy,252,1805


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 159 ms, sys: 35.4 ms, total: 195 ms
Wall time: 1.1 s


name
Alice       26098
Bob         22577
Charlie     23441
Dan         11840
Edith       10690
Frank       13625
George       5400
Hannah      24171
Ingrid      40361
Jerry        8904
Kevin       14953
Laura       16226
Michael     13674
Norbert     25711
Oliver       8337
Patricia    11942
Quinn       14750
Ray         32162
Sarah       16150
Tim         11692
Ursula      12504
Victor      26871
Wendy       42259
Xavier      20389
Yvonne      35871
Zelda        9402
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()

/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/mi